In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/ML team project/Project 2/Neubauer Chamber Automation


Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/12swurGjFVTSgi6fpziRcJ-tqsj6OoBgK/ML team project/Project 2/Neubauer Chamber Automation


In [ ]:
from skimage.transform import rotate, AffineTransform, warp
from skimage.filters import gaussian
import cv2
import numpy as np
import matplotlib.pyplot as plt

import os

#Neubauer Cell and Mask paths
NEU_CELL_PATH = 'Data Set/Originals/Images/'
NEU_MASK_PATH = 'Data Set/Originals/Masks/'

In [ ]:
#Get the paths to the original dataset
cell_paths = sorted(os.listdir(NEU_CELL_PATH))
mask_paths = os.listdir(NEU_MASK_PATH)
mask_paths = sorted([filename.replace('label','') for filename in mask_paths])

#Make sure they match each other
for cp, mp in zip(cell_paths, mask_paths):
    if cp != mp:
        print(cp, mp)

# Data Augmentation for the UNet

This is the code used to augment the data set used to train the UNet

In [ ]:
#Function used to turn the masks to binary
def threshold(img):
    out = img.copy().astype(np.uint8)
    #Threshold
    thresh = out.max()//2
    out[out<thresh] = 0
    out[out>=thresh] = 255

    #Some images are already grayscale, some aren't, so we check here
    if len(out.shape)>2:    
        out = cv2.cvtColor(out, cv2.COLOR_BGR2GRAY)
    #Sometimes after grayscale conversion we don't have a binary mask
    #this makes sure we do
    out[out>0] = 255
    return out

Here we do the actual data augmentation, and save the new images to the `train_augmented/` folder

In [ ]:
#Where we save the new images to
cell_out = 'Data Set/train_augmented/images/'
mask_out = 'Data Set/train_augmented/masks/'
i = 1
# Iterate over the original dataset
for cp, mp in zip(cell_paths, mask_paths):
    print(cp, mp)
    filename, ext = os.path.splitext(mp)
    mp = filename+'label'+ext
    
    # The first image has a bad mask, so we just skip over it entirely
    if cp == '1.PNG':
        continue
    
    #Load original image
    cell = cv2.imread(NEU_CELL_PATH + cp)
    mask = cv2.imread(NEU_MASK_PATH + mp)

    #JPEGs are special, and the masks are inversed to we fix them here
    if ext in ['.jpeg','.jpg']:
        mask = 255-mask

    # The second image has bad mask at the very top, so we just trim it
    if cp == '2.PNG':
        cell = cell[137:]
        mask = mask[137:]

    #Turn mask to binary and save original images
    mask = threshold(mask)
    cv2.imwrite(cell_out + f'{i}.jpg', cell)
    cv2.imwrite(mask_out + f'{i}.jpg', mask)
    i+=1
    
    #Rotated 45º
    cv2.imwrite(cell_out+f'{i}.jpg', (rotate(cell,angle=45,mode='wrap')*255).astype('uint8'))
    cv2.imwrite(mask_out+f'{i}.jpg', threshold(rotate(mask,angle=45,mode='wrap')*255))
    i+=1

    #Flipped left-right
    cv2.imwrite(cell_out+f'{i}.jpg', np.fliplr(cell))
    cv2.imwrite(mask_out+f'{i}.jpg', np.fliplr(mask))
    i+=1

    #Flipped up-down
    cv2.imwrite(cell_out+f'{i}.jpg', np.flipud(cell))
    cv2.imwrite(mask_out+f'{i}.jpg', np.flipud(mask))
    i+=1

    #Blurred
    cv2.imwrite(cell_out+f'{i}.jpg', (gaussian(cell,sigma=5,multichannel=True)*255).astype('uint8'))
    cv2.imwrite(mask_out+f'{i}.jpg', mask)
    i+=1

    #Darker
    cv2.imwrite(cell_out+f'{i}.jpg', (255*np.power(cell/255.0, 1.5)).astype('uint8'))
    cv2.imwrite(mask_out+f'{i}.jpg', mask)
    i+=1

    #Brighter
    cv2.imwrite(cell_out+f'{i}.jpg', (255*np.power(cell/255.0, 0.7)).astype('uint8'))
    cv2.imwrite(mask_out+f'{i}.jpg', mask)
    i+=1

    #### Compositions
    #Flipped LR + UD
    cv2.imwrite(cell_out+f'{i}.jpg', np.flipud(np.fliplr(cell)))
    io.imsave(mask_out+f'{i}.jpg', np.flipud(np.fliplr(mask)))
    i+=1

    # Rotated + blurred
    cv2.imwrite(cell_out+f'{i}.jpg', (gaussian(rotate(cell,angle=45,mode='wrap'),sigma=5,multichannel=True)*255).astype('uint8'))
    cv2.imwrite(mask_out+f'{i}.jpg', threshold(rotate(mask,angle=45,mode='wrap')*255))
    i+=1
    # Rotated + LR
    cv2.imwrite(cell_out+f'{i}.jpg', np.fliplr(rotate(cell,angle=45,mode='wrap')*255).astype('uint8'))
    cv2.imwrite(mask_out+f'{i}.jpg', threshold(np.fliplr(rotate(mask,angle=45,mode='wrap')*255)))
    i+=1

    # Rotated + UD
    cv2.imwrite(cell_out+f'{i}.jpg', np.flipud(rotate(cell,angle=45,mode='wrap')*255).astype('uint8'))
    cv2.imwrite(mask_out+f'{i}.jpg', threshold(np.flipud(rotate(mask,angle=45,mode='wrap')*255)))
    i+=1



1.PNG 1.PNG
10.PNG 10.PNG
11.PNG 11.PNG
12.PNG 12.PNG
13.jpeg 13.jpeg
14.jpeg 14.jpeg
15.jpeg 15.jpeg
16.PNG 16.PNG
17.PNG 17.PNG
18.PNG 18.PNG
19.PNG 19.PNG
2.PNG 2.PNG
20.PNG 20.PNG
21.PNG 21.PNG
22.PNG 22.PNG


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: UserWarning: ../cell-data/datasets/instagram/train_augmented/masks/151.jpg is a low contrast image


23.PNG 23.PNG
24.PNG 24.PNG
25.PNG 25.PNG
26.PNG 26.PNG
27.PNG 27.PNG
28.PNG 28.PNG
29.PNG 29.PNG
3.PNG 3.PNG
30.PNG 30.PNG
31.PNG 31.PNG
32.PNG 32.PNG
33.PNG 33.PNG
34.PNG 34.PNG
35.PNG 35.PNG
36.PNG 36.PNG
37.PNG 37.PNG


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: UserWarning: ../cell-data/datasets/instagram/train_augmented/masks/327.jpg is a low contrast image


38.PNG 38.PNG
39.PNG 39.PNG
4.PNG 4.PNG
40.PNG 40.PNG
41.PNG 41.PNG
42.PNG 42.PNG
43.PNG 43.PNG
5.PNG 5.PNG
53.PNG 53.PNG
54.PNG 54.PNG
55.PNG 55.PNG
56.PNG 56.PNG
57.PNG 57.PNG
58.PNG 58.PNG


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: UserWarning: ../cell-data/datasets/instagram/train_augmented/masks/481.jpg is a low contrast image


59.PNG 59.PNG


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: UserWarning: ../cell-data/datasets/instagram/train_augmented/masks/492.jpg is a low contrast image


6.PNG 6.PNG
60.PNG 60.PNG
61.PNG 61.PNG
7.PNG 7.PNG
77.PNG 77.PNG
8.PNG 8.PNG
9.PNG 9.PNG


# Data Augmentation for the Square Detection

This is the code used to get extra data for the Square Detection. Here we ignore the masks since we don't use them for the actual square detection. We do not use Rotation here either, as the algorithm can't detected rotated squares :(.
These images are saved to the `train_augmented/images/no-rotation/` folder

In [ ]:
cell_out = 'Data set/Square detection/Cropped-augmented-norotation/'

#Iterate over original images
for cp in cell_paths:
    i, ext = os.path.splitext(cp)
    print(cp, i)
    
    #Original
    cell = cv2.imread(NEU_CELL_PATH + cp)
    cv2.imwrite(cell_out + f'{i}_original.png', cell)
    
    #Flipped left-right
    cv2.imwrite(cell_out+f'{i}_flippedLR.png', np.fliplr(cell))
    
    #Flipped up-down
    cv2.imwrite(cell_out+f'{i}_flippedUD.png', np.flipud(cell))
   
    #Blurred
    cv2.imwrite(cell_out+f'{i}_blurred.png', (gaussian(cell,sigma=5,multichannel=True)*255).astype(np.uint8))

    #Darker
    cv2.imwrite(cell_out+f'{i}_darker.png', 255*np.power(cell/255.0, 1.5))
    
    #Brighter
    cv2.imwrite(cell_out+f'{i}_brighter.png', 255*np.power(cell/255.0, 0.7))
    
    #### Compositions
    #Flipped LR + UD
    cv2.imwrite(cell_out+f'{i}_flippedLR+UD.png', np.flipud(np.fliplr(cell)))


1.PNG 1
10.PNG 10
11.PNG 11
12.PNG 12
13.png 13
14.png 14
15.png 15
16.PNG 16
17.PNG 17
18.PNG 18
2.PNG 2
20.PNG 20
21.PNG 21
23.PNG 23
24.PNG 24
25.PNG 25
26.PNG 26
27.PNG 27
28.PNG 28
29.png 29
30.png 30
31.PNG 31
32.PNG 32
33.PNG 33
34.png 34
35.PNG 35
36.png 36
37.png 37
38.png 38
4.PNG 4
40.png 40
41.png 41
43.png 43
5.PNG 5
53.png 53
54.png 54
55.png 55
58.PNG 58
6.PNG 6
60.png 60
61.png 61
7.PNG 7
8.PNG 8
9.PNG 9
